In [8]:
import pandas

DATA_PATH_TWEET = r'./annotated data/merged_tweets.csv'

annotated_tweets = pandas.read_csv(DATA_PATH_TWEET, header=None, names = ['tweet', 'label'])

In [16]:
from sklearn.model_selection import train_test_split

X, y = annotated_tweets['tweet'], annotated_tweets['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.2, train_size=0.8)

In [10]:
import nltk

tokenizer = nltk.casual.TweetTokenizer(preserve_case=False, reduce_len=True)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(lowercase=True,
                             ngram_range=(1, 3),
                             analyzer='word',
                             tokenizer=tokenizer.tokenize) 

In [12]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

classifier = OneVsRestClassifier(LinearSVC(random_state=3))

In [13]:
from sklearn.pipeline import Pipeline

sentiment_pipeline = Pipeline([
        ('vectorizer', count_vect),
        ('classifier', classifier)
    ])

In [17]:
from sklearn.metrics import accuracy_score

sentiment_fit = sentiment_pipeline.fit(X_train, y_train)

y_pred = sentiment_fit.predict(X_test)
    
accuracy = accuracy_score(y_test, y_pred)

In [18]:
print(accuracy)

0.7936066713


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

classifier_log = LogisticRegression()

sentiment_pipeline_log_regr = Pipeline([
        ('vectorizer', count_vect),
        ('classifier', classifier_log)
    ])

In [23]:
from sklearn.metrics import accuracy_score

sentiment_fit_reg = sentiment_pipeline_log_regr.fit(X_train, y_train)

y_pred_reg = sentiment_fit_reg.predict(X_test)
    
accuracy_reg = accuracy_score(y_test, y_pred_reg)

In [24]:
print(accuracy_reg)

0.774496177901


In [ ]:

def train_test_and_evaluate(pipeline, x_train, y_train, x_test, y_test):
    if len(x_test[y_test == 0]) / (len(x_test)*1.) > 0.5:
        null_accuracy = len(x_test[y_test == 0]) / (len(x_test)*1.)
    else:
        null_accuracy = 1. - (len(x_test[y_test == 0]) / (len(x_test)*1.))
    sentiment_fit = pipeline.fit(x_train, y_train)
    y_pred = sentiment_fit.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    conmat = np.array(confusion_matrix(y_test, y_pred, labels=[0,1]))
    confusion = pd.DataFrame(conmat, index=['negative', 'positive'],
                         columns=['predicted_negative','predicted_positive'])
    print "null accuracy: {0:.2f}%".format(null_accuracy*100)
    print "accuracy score: {0:.2f}%".format(accuracy*100)
    if accuracy > null_accuracy:
        print "model is {0:.2f}% more accurate than null accuracy".format((accuracy-null_accuracy)*100)
    elif accuracy == null_accuracy:
        print "model has the same accuracy with the null accuracy"
    else:
        print "model is {0:.2f}% less accurate than null accuracy".format((null_accuracy-accuracy)*100)
    print "-"*80
    print "Confusion Matrix\n"
    print confusion
    print "-"*80
    print "Classification Report\n"
    print classification_report(y_test, y_pred, target_names=['negative','positive'])
